In [23]:
import pandas as pd 
import numpy as np
import hashlib
from csv import reader
import re
import os
pd.set_option('display.max_colwidth', None)

In [24]:
def hash_str(to_hash): 
    stringToHash = to_hash.encode()
    return hashlib.sha256(stringToHash).hexdigest()

In [25]:
def process_lbl(line_list, filename): 
    data_list = []
    i = 0
    name_line = ''
    while (i < len(line_list)): 
        if i+1 < len(line_list) and '---------,---------,---------,---------' in line_list[i+1]:
            name_line = line_list[i].strip('\n')     
        elif "---------,---------,---------,---------" in line_list[i]: 
            pass
        else: 
            for line in reader([filename+','+name_line+','+line_list[i].strip()]):
                data_list.append(line)
        i = i + 1
    return data_list
            
        
        

In [26]:
data = []
directory = 'tests/results'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if 'Data' not in filename and os.path.isfile(filepath):
        f = open(filepath, "r")
        lbl = [x for x in f]
        f.close()
        data.extend(process_lbl(lbl,filename))

In [27]:
df = pd.DataFrame(data, columns = ['app','doc','setup','time','regex','n_transitions','n_states',
                                    'test_type','component','test','val','metric'])
df.val = df.val.astype(int)


In [39]:
df.loc[(df.app=='pihole') & (df.metric=='μs') & (df.test=="prove+wit")]

,app,doc,setup,time,regex,n_transitions,n_states,test_type,component,test,val,metric
12337,pihole,ad.stackov_128,safa+nlookup,1700524121,"^ad([sxv]?[0-9]*|system)[_.-]([^.[:space:]]+\.){1,}|[_.-]ad([sxv]?[0-9]*|system)[_.-]",3229,28,R,P,prove+wit,2265605,μs
12358,pihole,adimage101_128,safa+nlookup,1700524145,^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-],4007,36,R,P,prove+wit,1914823,μs
12374,pihole,adimage101_128,safa+nlookup,1700524169,^(.+[_.-])?telemetry[_.-],3233,30,R,P,prove+wit,1611325,μs
12403,pihole,adimage101_128,safa+nlookup,1700524191,^adim(age|g)s?[0-9]*[_.-],1420,12,R,P,prove+wit,1900094,μs
12412,pihole,adtracker1_128,safa+nlookup,1700524213,^adtrack(er|ing)?[0-9]*[_.-],1807,15,R,P,prove+wit,1913135,μs
...,...,...,...,...,...,...,...,...,...,...,...,...
54757,pihole,beacons221_128,nwr,1700895314,^beacons?[0-9]*[_.-],1280,10,R,P,prove+wit,35926389,μs
55025,pihole,counters22_128,nwr,1700895418,^count(ers?)?[0-9]*[_.-],1408,11,R,P,prove+wit,36521900,μs
55293,pihole,mads.testi_128,nwr,1700895562,^mads\.,896,7,R,P,prove+wit,35572612,μs
55553,pihole,pixel.test_124,nwr,1700895695,^pixels?[-.],1152,9,R,P,prove+wit,34076695,μs


In [52]:
df.loc[
    (df.app=='pihole') & 
    (df.metric=='μs') & 
    (df.test!="prove+wit") &
    (df.component=='P')].groupby(by=['regex','setup','time']).sum()

val
regex                                     setup        time               
^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-] naive        1700754345  7360723
                                                       1700768295  7562112
                                                       1700776775  7480129
                                                       1700783761  7413330
                                                       1700793410  7377623
...                                                                    ...
^stat(s|istics)?[0-9]*[_.-]               safa+nlookup 1700524406  2124228
                                                       1700540879  2180467
                                                       1700576386  2116854
                                                       1700617818  2182916
                                                       1700657616  2217168

[490 rows x 1 columns]

In [46]:
df.loc[(df.app=='pihole') & (df.metric=='μs') & (df.test=="prove+wit")].groupby(by=['regex','setup']).std().val/1e6

regex                                                                                  setup       
^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-]                                              nwr             0.284127
                                                                                       reef            0.063768
                                                                                       safa+nlookup    0.039328
^(.+[_.-])?telemetry[_.-]                                                              nwr             0.279364
                                                                                       reef            0.031341
                                                                                       safa+nlookup    0.024067
^ad([sxv]?[0-9]*|system)[_.-]([^.[:space:]]+\.){1,}|[_.-]ad([sxv]?[0-9]*|system)[_.-]  nwr             0.451661
                                                                                       reef            0.055088
    

In [47]:
df.loc[(df.app=='pihole') & (df.metric=='μs') & (df.test=="prove+wit")].groupby(by=['regex','setup']).mean()/1e6

val
regex                                                                                 setup                   
^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-]                                             nwr            48.600932
                                                                                      reef            1.995572
                                                                                      safa+nlookup    1.968505
^(.+[_.-])?telemetry[_.-]                                                             nwr            42.901848
                                                                                      reef            1.596752
                                                                                      safa+nlookup    1.589398
^ad([sxv]?[0-9]*|system)[_.-]([^.[:space:]]+\.){1,}|[_.-]ad([sxv]?[0-9]*|system)[_.-] nwr           150.977505
                                                                                      reef            2.166417
                                                                                      safa+nlookup    2.327047
^adim(age|g)s?[0-9]*[_.-]                                                             nwr            36.415647
                                                                                      reef            1.821713
                                                                                      safa+nlookup    1.896485
^adtrack(er|ing)?[0-9]*[_.-]                                                          nwr            37.467548
                                                                                      reef            1.822239
                                                                                      safa+nlookup    1.936215
^advert(s|is(ing|ements?))?[0-9]*[_.-]                                                nwr            39.688132
                                                                                      reef            1.810240
                                                                                      safa+nlookup    1.918431
^aff(iliat(es?|ion))?[_.-]                                                            nwr            38.076186
                                                                                      reef            1.781811
                                                                                      safa+nlookup    1.839414
^analytics?[_.-]                                                                      nwr            36.633633
                                                                                      reef            1.815151
                                                                                      safa+nlookup    1.930602
^banners?[_.-]                                                                        nwr            35.794135
                                                                                      reef            1.803497
                                                                                      safa+nlookup    1.863593
^beacons?[0-9]*[_.-]                                                                  nwr            35.919674
                                                                                      reef            1.804268
                                                                                      safa+nlookup    1.897656
^count(ers?)?[0-9]*[_.-]                                                              nwr            36.304905
                                                                                      reef            1.794332
                                                                                      safa+nlookup    1.884559
^mads\.                                                                               nwr            35.081094
                                                                                      reef            1.507168


In [48]:
df.loc[(df.app=='pihole') & (df.metric=='μs') & (df.test=="prove+wit")].groupby(by=['regex','setup']).var()/1e6

val
regex                                                                                 setup                      
^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-]                                             nwr            80727.967525
                                                                                      reef            4066.411863
                                                                                      safa+nlookup    1546.692280
^(.+[_.-])?telemetry[_.-]                                                             nwr            78044.202670
                                                                                      reef             982.232551
                                                                                      safa+nlookup     579.242881
^ad([sxv]?[0-9]*|system)[_.-]([^.[:space:]]+\.){1,}|[_.-]ad([sxv]?[0-9]*|system)[_.-] nwr           203997.244248
                                                                                      reef            3034.654088
                                                                                      safa+nlookup    3244.378238
^adim(age|g)s?[0-9]*[_.-]                                                             nwr           181331.210487
                                                                                      reef            3297.437841
                                                                                      safa+nlookup     517.831524
^adtrack(er|ing)?[0-9]*[_.-]                                                          nwr           105924.777146
                                                                                      reef            1452.280015
                                                                                      safa+nlookup    1111.536812
^advert(s|is(ing|ements?))?[0-9]*[_.-]                                                nwr            86990.646591
                                                                                      reef             696.236094
                                                                                      safa+nlookup     128.992361
^aff(iliat(es?|ion))?[_.-]                                                            nwr            77261.436944
                                                                                      reef            2505.888913
                                                                                      safa+nlookup    1879.906302
^analytics?[_.-]                                                                      nwr            71996.378023
                                                                                      reef             786.685033
                                                                                      safa+nlookup    1720.746114
^banners?[_.-]                                                                        nwr            85478.450238
                                                                                      reef            1010.452535
                                                                                      safa+nlookup    2114.356407
^beacons?[0-9]*[_.-]                                                                  nwr           139054.849786
                                                                                      reef            1668.038727
                                                                                      safa+nlookup    2669.213159
^count(ers?)?[0-9]*[_.-]                                                              nwr            39270.816296
                                                                                      reef            1846.467144
                                                                                      safa+nlookup    1518.331564
^mads\.                                                                               nwr           130859.016473
      